In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 5, a_d_init = 2,
    b_d_init = 25, a_s_init = 2, b_s_init = -3, Dflag_init = True, Sflag_init = False):
        
    # Declare widgets for interactive input
    Qmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $Q$:',
                                 value = Qmax_init,
                                 continuous_update =True)
    Pmax_slider = widgets.IntSlider(min=5,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $P$:',
                                 value = Pmax_init,
                                 continuous_update =True)
    Qval_slider = widgets.FloatSlider(min=0.001,
                                 max=15,
                                 description='Pick a Quantity:',
                                 value = Qval_init,
                                 continuous_update = True)
    a_d_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_d$:',
                                 value = a_d_init,
                                 continuous_update =True)
    b_d_slider = widgets.FloatSlider(min=0,
                                 max=1000,
                                 description= r'$b_d$:',
                                 value=b_d_init,
                                 continuous_update =True)
    a_s_slider = widgets.FloatSlider(min=0.001,
                                 max=100,
                                 description= r'$a_s$:',
                                 value = a_s_init,
                                 continuous_update =True)
    b_s_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description= r'$b_s$:',
                                 value = b_s_init,
                                 continuous_update =True)
    Dflag_check = widgets.Checkbox(value = Dflag_init,
                                   description='Include Demand',
                                   disabled=False,
                                   indent=True) 
    Sflag_check = widgets.Checkbox(value = Sflag_init,
                                   description='Include Supply',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((Qmax_slider,'value'),(Qval_slider,'max'))
    widgets.jslink((b_d_slider,'value'),(b_s_slider,'max'))

    def elasticity_plot(Qmax, Pmax, Qval, a_d, b_d, a_s, b_s, Dflag, Sflag):

        # create a quantity vector, calculate supply/demand vectors
        Q = np.arange(0,Qmax)
        P_s = a_s*Q + b_s
        P_d = -a_d*Q + b_d

        # Calculate equilibrium quantity/price
        Qeq = (b_d-b_s)/(a_s + a_d)
        Peq = a_s*Qeq + b_s

        # Calculate prices for selected value
        Pval_s = a_s*Qval + b_s
        Pval_d = -a_d*Qval + b_d

        # Create figure, plot supply/demand
        fig, ax = plt.subplots(figsize=(20,10))
        if Sflag is True:
            ax.plot(Q, P_s,'r', linewidth=2, alpha=0.6,
                    label=r'Inverse Supply $\quad P_s = a_s Q + b_s$')
        if Dflag is True:
            ax.plot(Q, P_d,'b', linewidth=2, alpha=0.6,
                    label=r'Inverse Demand $\quad P_d = -a_d Q + b_d$')

        # Add markers for the price/quantity points, with dotted lines
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        if Sflag is True and Dflag is True:
            ax.scatter(Qeq, Peq, s=mrkrSize, c='k', label='Equilibrium')       
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6)

            ax.plot([Qeq,Qeq],[0,Peq],'k--',linewidth=1)
            ax.plot([0,Qeq],[Peq,Peq],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,max(Pval_s,Pval_d)],'k--',linewidth=1)
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.annotate(r'$Q^*$',[Qeq,0], xytext = [Qeq+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P^*$',[0,Peq], xytext = [0.15,Peq+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is True and Dflag is False:
            ax.scatter(Qval, Pval_s, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_s,Pval_s],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_s],'k--',linewidth=1)    
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_s$',[0,Pval_s], xytext = [0.15,Pval_s+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        elif Sflag is False and Dflag is True:
            ax.scatter(Qval, Pval_d, s=mrkrSize, c='k', alpha=0.6, 
                       label='Selection')
            ax.plot([0,Qval],[Pval_d,Pval_d],'k--',linewidth=1)
            ax.plot([Qval,Qval],[0,Pval_d],'k--',linewidth=1)
            ax.annotate(r'$Q$',[Qval,0], xytext = [Qval+0.15,0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)
            ax.annotate(r'$P_d$',[0,Pval_d], xytext = [0.15,Pval_d+0.25], 
                        xycoords ='data', fontsize = 25, clip_on = True)

        # Add elasticity annotations
        if Sflag is True:
            EsStr = r'$E_p^S = {:.2f}$'.format(Pval_s/(Qval*a_s))
            ax.annotate(EsStr,[Qval,Pval_s],
                        xytext = [Qval+0.25,Pval_s-1], 
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)
        if Dflag is True:
            EdStr = r'$E_p^D = {:.2f}$'.format(-Pval_d/(Qval*a_d))
            ax.annotate(EdStr,[Qval,Pval_d],
                        xytext = [Qval+0.25,Pval_d],
                        xycoords ='data',
                        fontsize = 25,
                        clip_on = True)

        # Add legend and format axes to look nice
        if Sflag is True or Dflag is True:
            ax.legend(loc='upper center', frameon=False,prop={'size':20})
        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = Pmax, bottom = 0)
        ax.set_xlim(right = Qmax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$Q$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$P$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(elasticity_plot, {'Qmax': Qmax_slider,
                                                       'Pmax': Pmax_slider,
                                                       'Qval': Qval_slider, 
                                                       'a_d': a_d_slider,
                                                       'b_d': b_d_slider,
                                                       'a_s': a_s_slider, 
                                                       'b_s': b_s_slider, 
                                                       'Dflag': Dflag_check,
                                                       'Sflag': Sflag_check})

    output = widgets.VBox([out,
                  widgets.HBox([Qval_slider,
                               Qmax_slider,
                               Pmax_slider]),
                  widgets.HBox([Dflag_check, 
                                a_d_slider, 
                                b_d_slider]),
                  widgets.HBox([Sflag_check, 
                                a_s_slider, 
                                b_s_slider])])
    display(output)


In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Elasticities: An application of linear functions in economics

In [3]:
Elasticities_widget()

Blah Blah Example of an 'inelastic' market. Note that the elasticities *still change* on the curves if you check the quantities

In [4]:
Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 3.3, a_d_init = 75,
    b_d_init = 275, a_s_init = 50, b_s_init = -150, Dflag_init = True, Sflag_init = True)

Blah Blah Elastic this time

In [5]:
Elasticities_widget(Qmax_init = 15, Pmax_init = 30, Qval_init = 6, a_d_init = 0.5,
    b_d_init = 22, a_s_init = 0.75, b_s_init = 10, Dflag_init = True, Sflag_init = True)